<a href="https://isaic.ece.ualberta.ca/"> <center><img src="img/index.png" alt="Header" style="width: 200px;"/></center> </a>

> # Accelerated Python Programming using Numba
>
> **Presentor:** Industry Sandbox and AI Computing (ISAIC)
>
> **Date:** 

- **[Numba](http://numba.pydata.org/)** is a just-in-time python function compiler that accelerates numerically-focused python functions for both CPUs and GPUs. Numba uses a collection of decorators that apply on python functions when called for execution. The compiler converts normal python functions to (usually) faster python functions in machine code.

- **How does Numba work?**

<center><img src="img/numba1.PNG" alt="Header" style="width: 400px;"/></center>


## Introduction to Numba for CPU

The most fundamental decorator to compile python functions is `@jit`. Let's see how it works...

In [1]:
from numba import jit
import math

# This is the function decorator syntax and is equivalent to `doublegauss = jit(doublegauss)`.
# The Numba compiler is just a function you can call whenever you want!
@jit
def doublegauss(x, y):
    '''For given x and y values, computes the gaussian function'''
    #the following few lines are redundant, but used to keep the computation a bit more lengthy
    x = abs(x)
    y = abs(y)
    t1 = x*x
    t2 = y*y
    val = math.exp(-t1-t2)
    return val

In [3]:
#compiler converts python function into machine code
doublegauss(1,2)

#numba also saves the original python function
doublegauss.py_func(1,2)

0.006737946999085467

In [4]:
#let's see how the control-flow graph object looks like
print (doublegauss.inspect_cfg(doublegauss.signatures[0]))

digraph "CFG for '_ZN8__main__15doublegauss_241B46c8tJTIeFCjyCbUFRqqOAFv_2fYRdE1AT0EZmkCAA_3d_3dExx' function" {
	label="CFG for '_ZN8__main__15doublegauss_241B46c8tJTIeFCjyCbUFRqqOAFv_2fYRdE1AT0EZmkCAA_3d_3dExx' function";

	Node0x5640e2c7f1b0 [shape=record,label="{entry:\l  %.13 = mul i64 %arg.y, %arg.y\l  %0 = mul i64 %arg.x, %arg.x\l  %.14 = sub i64 0, %0\l  %.15 = sub nsw i64 %.14, %.13\l  %.16 = sitofp i64 %.15 to double\l  %.17 = tail call double @llvm.exp.f64(double %.16)\l  store double %.17, double* %retptr, align 8\l  ret i32 0\l}"];
}



- **Benchmarking runtime:**

In order to validate how effective Numba compiler is, we need to measure the performance of each run of the code. For this we will use `%timeit` magic function that computes the runtime many times and reports an average value.

In [3]:
#use the magic function %timeit

#for original python function implementation
%timeit doublegauss.py_func(1,2)

300 ns ± 0.203 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
#for numba compiled python function
%timeit doublegauss(1,2)

232 ns ± 0.183 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


We see that numba is already faster on CPU compared to usual python function run. This is because, once the numba compiler finished compiling the python function into machine code, it re-uses the pre-compiled object to execute without any overhead. Where as, the usual python function compiler starts compiling from scratch everytime the function is called.

- **A bit more heavier task in python function**

Although we have seen a slight improvement, our function is too simple to draw any significant conclusion from the above example.

Let's look into another function example that uses a heavier computational task.

In this example, we will use Monte Carlo method to determine the value of Pi. The way we do this is by drawing two random numbers independently between [0,1] and check if the point lies inside 1/4th of a circle with unit radius. The diagram below visualizes the problem of calculating value of Pi:

<center><img src="img/mc.PNG" alt="Header" style="width: 400px;"/></center>

Now higher the number of random (x,y) samples we draw, the more precise value of Pi we will get. Also the computation work increases with the increase of the number of the random samples.

In [10]:
#set the number of (x,y) samples to draw
nsamples = 1000000

import random

@jit
def monte_carlo_pi(nsamples):
    ''' Calculates the value of Pi'''
    #counter for the points that lie within the circle
    acc = 0
    #iterate over the number of samples
    for i in range(nsamples):
        #draw independent random samples everytime
        x = random.random()
        y = random.random()
        #check if the point lies within the circle
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [8]:
%timeit monte_carlo_pi(nsamples)

11.9 ms ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit monte_carlo_pi.py_func(nsamples)

394 ms ± 604 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


That's more than **30 times faster** than normal python compiler! So just by adding single line of Numba decorator can increase the performance of python computation significantly.

- **Limited data type support:**

Numba doesn't support certain data types, (e.g. dictionaries). However by default, numba falls back to python **object mode** in such scenarios. To force implement numba compilation, we can use additional argument in the decorator `nopython=True` to force run in **nopython mode**

In [11]:
@jit
def dict_func(x):
    return x['key']

#without mode specification, it will only give a warning message
#but will the run the function after falling back to object mode

dict_func(dict(key='value'))

/tmp/ipykernel_277036/324830447.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "dict_func" failed type inference due to: non-precise type pyobject
During: typing of argument at /tmp/ipykernel_277036/324830447.py (3)

File "../../../tmp/ipykernel_277036/324830447.py", line 3:
<source missing, REPL/exec in use?>

  @jit
/opt/tljh/user/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "dict_func" was compiled in object mode without forceobj=True.

File "../../../tmp/ipykernel_277036/324830447.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/opt/tljh/user/lib/python3.9/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.readthedocs.io/en/stable/referen

'value'

In [12]:
@jit(nopython=True)
def dict_func(x):
    return x['key']

#since here we have specified to run in nopython mode, any given non-supported data type
#in function argument will raise error.
dict_func(dict(key='value'))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at /tmp/ipykernel_277036/3933219848.py (3)[0m
[1m
File "../../../tmp/ipykernel_277036/3933219848.py", line 3:[0m
[1m<source missing, REPL/exec in use?>[0m 

This error may have been caused by the following argument(s):
- argument 0: [1mCannot determine Numba type of <class 'dict'>[0m


While using Numba compiler for acceleration purpose, it is good practice to force run with `nopython` as it will ensure the implementation of Numba compilation.

## Numpy ufuncs computation on GPUs

GPU hardware is designed for *data parallelism*. Full potential is achieved when the GPU is computing the same operations on sufficiently large number of elements at once.

NumPy Universal functions (ufuncs) (e.g. `np.add`, `np.multiply` etc), which perform the same operation on every element in a NumPy array, are naturally data parallel, so they are a natural fit for GPU programming.

- **Some Terminology:**

Before we proceed further, let's introduce some terms that we will frequently use for the rest of the tutorial.

    - host : CPU
    - device : GPU
    - host memory : main system memory (RAM)
    - device memory : Onboard memory on GPU card (graphics memory)
    - kernel : a GPU function launched by the host and executed on the device
    - device function : a GPU function executed on the device and can only be called from the device

Unfortunately, NumPy in-built ufuncs are inherent to CPU and cannot be compiled with Numba. However, we can create our own ufunc using Numba compiler. We first create a python scalar function and then decorate it with Numba `vectorize`.

In this example we will use the `@vectorize` decorator to compile and optimize a ufunc for the CPU first.

In [13]:
import numpy as np
from numba import vectorize

@vectorize
def mult_ten(x):
    '''takes a scaler value x and mutliplies by 10'''
    return x*10
#the scalar value x operation will be performed on each numpy element

In [14]:
arr = np.arange(10)

mult_ten(arr)#passing the whole array to the python function

array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

- **Compiling on CUDA**

Let us now generate a ufunc that uses CUDA on the GPU with the addition of giving an **explicit type signature** and setting the `target` attribute. The type signature argument describes what types to use both for the ufuncs arguments and return value:
```python
'return_value_type(argument1_value_type, argument2_value_type, ...)'
```

Please see the Numba docs for more on [available types](https://numba.pydata.org/numba-doc/dev/reference/types.html), as well as for additional information on [writing ufuncs with more than one signature](https://numba.pydata.org/numba-doc/dev/user/vectorize.html)

Here is a simple example of a ufunc that will be compiled for a CUDA enabled GPU device. It expects two `int64` values and return also an `int64` value:

In [15]:
@vectorize(['int64(int64, int64)'], target='cuda') # Type signature and target are required for the GPU
def mult_func(x,y):
    return x*y

In [16]:
a = np.array([1,2,3,4,5,6])
b = np.array([10,20,30,40,50,60])
mult_func(a,b)

/opt/tljh/user/lib/python3.9/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (160) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([ 10,  40,  90, 160, 250, 360])

After calling numba gpu function,  Numba just automatically executed the follwoing steps in sequence:

 * Compiled a CUDA kernel to execute the ufunc operation in parallel over all the input elements.
 * Allocated GPU memory for the inputs and the output.
 * Copied the input data to the GPU.
 * Executed the CUDA kernel (GPU function) with the correct kernel dimensions given the input sizes.
 * Copied the result back from the GPU to the CPU.
 * Returned the result as a NumPy array on the host.


**Side note:** Taking a look at the data types can sometimes be important in GPU code because the performance of `float32` (single precision) and `float64` (double precision) computations can (depending on the GPU) be very different on CUDA devices. If your algorithm can obtain correct results using `float32`, then you should probably use that data type, because casting to `float64` can, depending on the GPU type, dramatically slow down the function.

- **Benchmarking the results**

In [18]:
#in-built numpy function
%timeit np.multiply(a,b)

497 ns ± 0.0933 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [19]:
#numbe compiled function executed on GPU
%timeit mult_func(a,b)

1.06 ms ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


The GPU function is **a lot slower** than the CPU! For this example, it is expected as we have misused GPU in several ways:

- Inputs are too small (cause for the above warning message)
- Calculation is too simple
- Data is copied to and from the GPU
- Data types are larger than necessary

It is important to understand which problems are well suited for GPU. Let us now try an example that is more intense in computation and actually faster in the GPU.

In [20]:
import math # Note that for the CUDA target, we need to use the scalar functions from the math module, not NumPy

SQRT_2PI = np.float32((2*math.pi)**0.5)  # Precompute this constant as a float32.

@vectorize(['float32(float32, float32, float32)'], target='cuda')
def gaussian_pdf(x, mean, sigma):
    '''Compute the value of a Gaussian probability density function at x with given mean and sigma.'''
    return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)

In [21]:
import numpy as np
# Evaluate the Gaussian a million times!
x = np.random.uniform(-3, 3, size=100000000).astype(np.float32)
mean = np.float32(0.0)
sigma = np.float32(1.0)

# Quick test on a single element just to make sure it works
gaussian_pdf(x[0], 0.0, 1.0)

/opt/tljh/user/lib/python3.9/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (160) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([0.0138177])

Note that we are still getting the warning message even after using large array `x` is because, when we called `gaussian_pdf` above it only used a single value from the array.

In [22]:
import scipy.stats # for definition of gaussian distribution, so we can compare CPU to GPU time
norm_pdf = scipy.stats.norm
%timeit norm_pdf.pdf(x, loc=mean, scale=sigma)

4.64 s ± 4.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit gaussian_pdf(x, mean, sigma)

470 ms ± 30.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


This is a big improvement (almost **10 times faster**). To compare with compiled CPU version, let's define the following function.

In [22]:
@vectorize
def cpu_gaussian_pdf(x, mean, sigma):
    '''Compute the value of a Gaussian probability density function at x with given mean and sigma.'''
    return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)

In [23]:
%timeit cpu_gaussian_pdf(x, mean, sigma)

1.48 s ± 4.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Comparision:**

- Uncompiled CPU scipy function: 4.6 s ± 5.41 ms
- Compiled custom CPU function: 1.48 s ± 4.45 ms
- Compiled GPU function: 460 ms ± 27.3 ms

## CUDA Device functions

To compile functions for the GPU that are **not** element wise, vectorized functions, we use `numba.cuda.jit`. Let's now see how to use it to decorate a helper function, to be utilized by a GPU accelerated ufunc, so that we are not required to write all our logic into a single ufunc defintion or having to write separate unfuncs with detailed decorator everytime.

Notice that `polar_to_cartesian` below does not require a type signature, and also, that it is passed two scalar values, unlike the vectorized ufuncs we have been using (and like `polar_distance` below) which expect NumPy arrays as arguments.

The argument `device=True` indicates that the decorated function can **only** be called from a function running on the GPU, and not from CPU host code:

In [29]:
from numba import cuda

@cuda.jit(device=True)
def polar_to_cartesian(rho, theta):
    x = rho * math.cos(theta)
    y = rho * math.sin(theta)
    return x, y

@vectorize(['float32(float32, float32, float32, float32)'], target='cuda')
def polar_distance(rho1, theta1, rho2, theta2):
    x1, y1 = polar_to_cartesian(rho1, theta1) # We can use device functions inside our GPU ufuncs
    x2, y2 = polar_to_cartesian(rho2, theta2)
    
    return ((x1 - x2)**2 + (y1 - y2)**2)**0.5

In [15]:
n = 1000000
rho1 = np.random.uniform(0.5, 1.5, size=n).astype(np.float32)
theta1 = np.random.uniform(-np.pi, np.pi, size=n).astype(np.float32)
rho2 = np.random.uniform(0.5, 1.5, size=n).astype(np.float32)
theta2 = np.random.uniform(-np.pi, np.pi, size=n).astype(np.float32)

In [16]:
polar_distance(rho1, theta1, rho2, theta2)

array([0.52712027, 0.68070417, 1.92223535, ..., 1.18200532, 1.54287882,
       2.079705  ])

## Optimizing GPU memory utilization

So far we have used NumPy arrays on the CPU as inputs and outputs to our GPU functions. Numba has been automatically transferring this data to the GPU for us so that it can be operated on by the GPU. With this implicit data transfer Numba, acting conservatively, will automatically transfer the data back to the CPU after processing. This process is a time intensive operation and can be a bottle neck of the computation.

So our goal is to **minimiza the data transfer between the host (CPU) and the device (GPU)**

In order to implement this, we need to prevent Numba's automatic data transfer and manually copying it back to the host when all the operations ar done.

The way to do this is to create **CUDA Device Arrays** and pass them to our GPU functions. Device arrays will not be automatically transfered back to the host after processing, and can be reused as we wish on the device before ultimately, and only if necessary, sending them, or parts of them, back to the host.

To demonstrate, let's create our example addition ufunc again:

In [26]:
@vectorize(['float32(float32, float32)'], target='cuda')
def mult_func(x, y):
    return x * y

n = 1000000
x = np.arange(n).astype(np.float32)
y = 2 * x

In [27]:
%timeit mult_func(x, y)  # Baseline performance with host arrays

4.69 ms ± 48.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The `numba.cuda` module includes a function that will copy host data to the GPU and return a CUDA device array. Note that below when we try to print the content of the device array, we only get information about the array, and not its actual contents. This is because the data is on the device, and we would need to transfer it back to the host in order to print its values.

In [18]:
from numba import cuda

x_device = cuda.to_device(x)
y_device = cuda.to_device(y)

print(x_device)
print(x_device.shape)
print(x_device.dtype)

(100000,)
float32


In [30]:
#Device arrays can be passed to CUDA functions just like NumPy arrays
%timeit mult_func(x_device, y_device)

902 µs ± 2.96 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Because x_device and y_device are already on the device, this benchmark is much faster (almost **2 times**).

Note that, if we don't assign the output into a variable, it gets sent back to the host CPU. On the other hand, if the output is assigned into a variable, it stays as CUDA device array. In order to do this, we can initialize a CUDA output array and pass it as the output variable when calling the function.

In [32]:
out_device = cuda.device_array(shape=(n,), dtype=np.float32)  # does not initialize the contents, like np.empty()
%timeit mult_func(x_device, y_device, out=out_device)

737 µs ± 650 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


When we want to bring a device array back to the host memory, we can use the `copy_to_host()` method:

In [33]:
out_host = out_device.copy_to_host()
print(out_host[:10])

[  0.   2.   8.  18.  32.  50.  72.  98. 128. 162.]


## Accelarating Neural Network Computation on GPU

Now that we know how to perform python function calculations on GPU in a memory optimized way, let's implement a hidden layer neural network calculation accelarated on GPU.

A hidden layer operation in our example consists of normalizing grey scale values, weighting each input for the linear transformation and applies activation function.

In [45]:
# Our hidden layer will contain 1M neurons.
n = 1000000

greyscales = np.floor(np.random.uniform(0, 255, n).astype(np.float32))
weights = np.random.normal(.5, .1, n).astype(np.float32)

from math import exp


@vectorize(['float32(float32)'], target='cuda')
def normalize(grayscales):
    return grayscales / 255

@vectorize(['float32(float32, float32)'], target='cuda')
def weigh(values, weights):
    return values * weights
        
@vectorize(['float32(float32)'], target='cuda')
def activate(values):
    return ( exp(values) - exp(-values) ) / ( exp(values) + exp(-values) )


#here we can define a usual python function and call GPU-accelerated ufuncs inside
def create_hidden_layer(n, greyscales, weights, exp, normalize, weigh, activate):
    
    #first convert the host arrays to device array
    grey_device = cuda.to_device(greyscales)
    weight_device = cuda.to_device(weights)

    #assign an empty device array for output
    norm_out = cuda.device_array(shape=(n,), dtype=np.float32)
    normalize(grey_device, out=norm_out)
    
    #
    weight_out = cuda.device_array(shape=(n,), dtype=np.float32)
    weigh(norm_out, weight_device, out=weight_out)
    
    activated = activate(weight_out)
    activated = activated.copy_to_host()
    
    return activated

In [47]:
%timeit create_hidden_layer(n, greyscales, weights, exp, normalize, weigh, activate)

8.54 ms ± 88.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Bonus: Writing Custom CUDA Kernel

So far we know how to use GPU-accelerated ufuncs, device functions and optimize data movement too speed up our code. The vectorized GPU-accelerated python functions are a type of CUDA kernels that automatically distributes the parallel computing on the device based on the size of the data array. However, in reality our problems do not always consists of only universal array functions. For example, in our last example, to apply different GPU-accelerated ufuncs we needed to write a normal python function that executes on the host and sends inner gpu functions to the device. We can improve this implementation by writing the entire above python function as CUDA kernel and execute the entire code on the device.

Another important aspect of writing our own custom CUDA kernel is that we can manage the parallel computing distribution ourselves via **execution configuration** which CUDA kernels require during its run. To understand how to configure CUDA kernel execution, let's look into the following diagram:

<center><img src="img/cuda.PNG" alt="Header" style="width: 400px;"/></center>

- **Thread Hierarchy**

When we launch a GPU function (CUDA kernel) it runs on many **threads** in parallel. A collection of threads forms a **block**. A CUDA kernel can run on several blocks, and for a given launch, the collection of all the blocks associated form a **grid**. When we launch a kernel with execution configuration, it defines the *number of blocks* in the grid and *number of threads* in each block. CUDA and Numba together provides some thread hierarchy variables that we can use while writing our kernel. The idea is to somehow map the **index of each element** in the dataset array to the **index of each thread** in the grid, so when we launch the kernel it processes elements of the dataset in threads in parallel. Let's now look at these variables:

- `gridDim.x` : number of blocks in the grid
- `blockIdx.x` : index of the current block within the grid (starts with 0)
- `blockDim.x` : number of threads in a block (all blocks contain the same number of threads in a grid)
- `threadIdx.x` : index of the current thread within a block (starts with 0 in each block)
- `cuda.grid()` : Numba provides unique thread index using the formula: `threadIdx.x + blockIdx.x * blockDim.x`
- `cuda.gridsize()` : Numba provided variable that returns the total number of threads in the grid, i.e. `gridDim.x * blockDim.x`

Using the above variables, we can now coordinate the parallel cmputing by mapping the dataset element index to unique thread index. Let's see how we can implement out first CUDA kernel...

In [39]:

from math import exp

#convert the previously used GPU vectorized functions into device functions
@cuda.jit(device=True)
def normalize(grayscales):
    return grayscales / 255

@cuda.jit(device=True)
def weigh(values, weights):
    return values * weights
        
@cuda.jit(device=True)
def activate(values):
    return ( exp(values) - exp(-values) ) / ( exp(values) + exp(-values) )


#here we can define the CUDA kernel

#CUDA kernel does NOT return anything, instead we provide an empty outut array in the function
#input argument

@cuda.jit
def create_hidden_layer_kernel(greyscales, weights, out_array):

    #here the variable extracts the current thread's index
    start_idx = cuda.grid(1)
    #here we extract the total number of threads in the grid
    stride = cuda.gridsize(1)
    
    #here we implement grid strid loop, i.e. the current thread works on data elements
    #in the dataset starting from start index and jumping every stride number of elements
    #to work on the next element until there's no more element left in the dataset
    for idx in range(start_idx, greyscales.shape[0], stride):
        out_array[idx] = activate(weigh(normalize(greyscales[idx]), weights[idx]))

In [40]:
#check the GPU lists
print (cuda.gpus)

<Managed Device 0>, <Managed Device 1>


In [50]:
# Our hidden layer will contain 1M neurons.
n = 1000000

#setting the arrays
greyscales = np.floor(np.random.uniform(0, 255, n).astype(np.float32))
weights = np.random.normal(.5, .1, n).astype(np.float32)

greyscales_d = cuda.to_device(greyscales)
weights_d = cuda.to_device(weights)

out = cuda.device_array_like(greyscales_d)

threads_per_block = 128
blocks_per_grid = 128


In [58]:
%timeit create_hidden_layer_kernel[blocks_per_grid, threads_per_block](greyscales_d, weights_d, out); cuda.synchronize()
print (out.copy_to_host())

858 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
[0.5060845  0.12321589 0.35860813 ... 0.10897925 0.34099004 0.40804127]
